# ALBERT for aptamer-pair classification

## Set dependancies

In [ ]:
!pip install transformers==4.9.1
#  In case prediction doesnt work Check:
#  https://github.com/huggingface/transformers/issues/8879 and BERT architecture

In [ ]:
import torch
import torch.nn as nn
import torch.onnx
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
import ruamel.yaml
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup

os.environ["TOKENIZERS_PARALLELISM"] = "false"
config_name = 'config.yaml'

## Load your dataset

In [ ]:
with open(config_name, 'r') as stream:
    try:
        yaml = ruamel.yaml.YAML()
        config = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
path_to_train_data = config['Datasets']['train']
path_to_val_data = config['Datasets']['val']
path_to_test_data = config['Datasets']['test']

df_train = pd.read_csv(path_to_train_data)
df_val = pd.read_csv(path_to_val_data)
df_test = pd.read_csv(path_to_test_data)

In [ ]:
print(df_train.shape, df_val.shape, df_test.shape)

(399600, 3)
(49950, 3)
(49950, 3)


In [ ]:
df_train.head()

,Sequence1,Sequence2,Label
0,ATCTGGCTAATTAAT,GCTAGCTACTCTCAG,0
1,AAGGAGATGGTTGAA,GAAGGGTCGTCTCCA,0
2,AGATGCAAACGCTGC,TCGAATCTATAGACA,0
3,GCTGTCACGAACTTG,GTTTGGAGGCTGATC,1
4,CGAGTCTGTCTCGCC,ATAAAGCAGAAAAAT,1


## Classes and functions

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model='albert-base-v2'):
        self.data = data  # pandas dataframe
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model, return_dict=False)  
        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sent1 = str(self.data.loc[index, 'Sequence1'])
        sent2 = str(self.data.loc[index, 'Sequence2'])

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.maxlen,  
                                      return_tensors='pt')  # Return torch.Tensor objects
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.data.loc[index, 'Label']
            return token_ids, attn_masks, token_type_ids, label  
        else:
            return token_ids, attn_masks, token_type_ids

In [ ]:
class Model(nn.Module):

    def __init__(self, bert_model="albert-base-v2", freeze_bert=False):
        super(Model, self).__init__()
        self.bert_layer = AutoModel.from_pretrained(bert_model)

        #  Fix the hidden-state size of the encoder outputs
        #  More information can be found https://huggingface.co/transformers/pretrained_models.html
        hidden_size = config['Model']['hidden_size']

        # Freeze bert layers and only train the classification layer weights
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False

        # Classification layer
        self.cls_layer = nn.Linear(hidden_size, config['Model']['number_labels'])
        self.dropout = nn.Dropout(p=config['Model'['dropout_rate']])

    @autocast()  # run in mixed precision
    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to identify sentence1 and sentence2
        '''

        # Feeding the inputs to the BERT-based model to obtain contextualized representations
        cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)

        # Feeding to the classifier layer the last layer hidden-state of the [CLS] token further processed by a
        # Linear Layer and a Tanh activation. The Linear layer weights were trained from the sentence order prediction (ALBERT) or next sentence prediction (BERT)
        # objective during pre-training.
        logits = self.cls_layer(self.dropout(pooler_output))

        return logits

In [ ]:
#seed to make results reproducible
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate_loss(net, device, criterion, dataloader):
    net.eval()

    mean_loss = 0
    count = 0

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            count += 1

    return mean_loss / count

In [ ]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    best_loss = np.Inf
    nb_iterations = len(train_loader)
    print_every = nb_iterations // config['Random']['print_every']  # print the training loss 5 times per epoch
    scaler = GradScaler()
    
    iters = []
    train_losses = []
    val_losses = []

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                logits = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # Backpropagating the gradients
            # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, opti.step() is then called,
                # otherwise, opti.step() is skipped.
                scaler.step(opti)
                # Updates the scale for next iteration.
                scaler.update()
                # Adjust the learning rate based on the number of iterations.
                lr_scheduler.step()
                # Clear gradients
                opti.zero_grad()


            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0


        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss

    # Saving the model
    path_to_model='model/{}_{}_{}.pt'.format(bert_model, lr, round(best_loss, 4))
    path_to_model_evaluation = './datasets/model_validation/{}_{}_{}.pt'.format(bert_model, lr, round(best_loss, 4))
    
    config['Random']['path_to_model'] = path_to_model
    config['Random']['path_to_model_evaluation'] = path_to_model_evaluation

    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()

## Hyperparameters for training

In [ ]:
bert_model = config['Model']['bert_model']  # any of previously defined BERT alternatives :'albert-base-v2', 'albert-large-v2', 'albert-xlarge-v2', 'albert-xxlarge-v2' and others
freeze_bert = config['Model']['freeze_bert']  # if True, freeze the encoder weights and only update the classification layer weights
maxlen = config['Model']['max_len']        # maximum length of the tokenized input sentence pair : if greater than "maxlen", the input is truncated and else if smaller, the input is padded
bs = config['Model']['batch_size']                # batch size
iters_to_accumulate = config['Model']['itters_to_accumulate']  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = config['Model']['learning_rate']                # learning rate
epochs = config['Model']['epochs']             # number of training epochs

## Training

In [ ]:
#  Set all seeds to make reproducible results
set_seed(2)
 
# Creating instances of training and validation set
print("Reading training data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
print("Reading validation data...")
val_set = CustomDataset(df_val, maxlen, bert_model)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=bs, num_workers=2)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=2)
 
 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Model(bert_model, freeze_bert=freeze_bert)

if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)
   
criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
num_warmup_steps = config['Model']['num_warmup_steps'] # The number of steps for the warmup phase.
num_training_steps = epochs * len(train_loader)  # The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)
                         
train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

with open('config.yaml', 'w') as conf:
    yaml.dump(config, conf)

Reading training data...
Reading validation data...


 10%|▉         | 624/6244 [02:16<20:26,  4.58it/s]


Iteration 624/6244 of epoch 1 complete. Loss : 0.35080221448189175 


 20%|█▉        | 1248/6244 [04:32<18:08,  4.59it/s]


Iteration 1248/6244 of epoch 1 complete. Loss : 0.32330357120969355 


...


Iteration 5616/6244 of epoch 20 complete. Loss : 0.1904088930489543 


100%|█████████▉| 6240/6244 [22:47<00:00,  4.55it/s]


Iteration 6240/6244 of epoch 20 complete. Loss : 0.18800144887362152 


100%|██████████| 781/781 [01:03<00:00, 12.25it/s]



Epoch 20 complete! Validation Loss : 0.4195267601408513
The model has been saved in models/albertLarge8.pt


In [ ]:
#!kill -9 -1 #in case you have to kill the process

## Prediction

In [ ]:
def get_probs_from_logits(logits):
    """
    Converts a tensor of logits into an array of probabilities by applying the sigmoid function
    """
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()


def test_prediction(net, device, aptamerDataFrame, dataloader, with_labels, result_path=config['Random']['path_to_model_evaluation']):
    """
    Predict the probabilities on a dataset with or without labels and print the result in a file
    """
    net.eval()
    probs_all = []
    nb_iterations = len(dataloader)
    
    with torch.no_grad():
        if with_labels:
            for it, (seq, attn_masks, token_type_ids) in tqdm(enumerate(dataloader), total = nb_iterations):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
                
        else:
            for it, (seq, attn_masks, token_type_ids) in tqdm(enumerate(dataloader), total=nb_iterations):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
            
    y_hat = [round(x) for x in probs_all]     
    df2 = pd.DataFrame({'y_hat': y_hat, 'prob': probs_all})
    df = pd.concat([aptamerDataFrame, df2], axis=1)
    df.to_csv(result_path)


In [ ]:
print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=4)

model = Model(bert_model)
if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

print()
print("Loading the weights of the model...")
model.load_state_dict(torch.load(path_to_model))
model.to(device)

print("Predicting on test data...")
test_prediction(net=model
                , device=device
                , dataloader=test_loader
                , aptamerDataFrame=df_train)

print()
print("Predictions are available in : {}".format(config['Random']['path_to_model_evaluation']))

Reading test data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/781 [00:00<?, ?it/s]


Loading the weights of the model...
Predicting on test data...


100%|██████████| 781/781 [01:03<00:00, 12.25it/s]



Predictions are available in : results/output.txt


#  Model Metrics

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv(config['Random']['path_to_model_evaluation'])

accuracy = accuracy_score(df['Label'], df['y_hat'])
precision = precision_score(df['Label'], df['y_hat'])
recall = recall_score(df['Label'], df['y_hat'])
f1 = f1_score(df['Label'], df['y_hat'])
roc_auc = roc_auc_score(df['Label'], df['prob'])
matrix = confusion_matrix(df['Label'], df['y_hat'])

print('accuracy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)
print('roc_auc: ', roc_auc)
print('matrix: \n', matrix)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve

fpr,tpr,threshold = roc_curve(df['Label'], df['prob'])
roc_auc =auc(fpr, tpr)

plt.plot(fpr, tpr, label='ROC curve')
plt.plot([0,1],[0,1], 'k--')
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.legend(loc='lower right')

#  Convert model to ONNX


In [ ]:
if config['Model']['convert_to_onnx']:
    sequences = {'AGCTAAGCTAAGCTA':'AAGACTGACAGCTAA'}
    sequences=pd.DataFrame(sequences.items(), columns=['Sequence1', 'Sequence2'])
    dataset = CustomDataset(
        data = sequences,
        maxlen = maxlen,
        with_labels = False #sulyginti su class cusotmedatasets
        )
    model = Model(bert_model)
    #model = nn.DataParallel(model)
    model.load_state_dict(torch.load(path_to_model))
    model.to(device)
    model.eval()

    #defining model inputs, which are ids, kmask and toke type ids (it comes from Model Class)
    input_ids= dataset[0]['token_ids'].unsqueeze(0).cuda()
    attn_masks = dataset[0]['attn_masks'].unsqueeze(0).cuda()
    token_type_ids = dataset[0]['token_type_ids'].unsqueeze(0).cuda()
        
    torch.onnx.export(
        model, #.module if paralized
        (input_ids, attn_masks, token_type_ids),
        "model.onnx",
        input_names=["input_ids", "attn_masks", "token_type_ids"], 
        output_names=["output"],
        #which inputs have dynamical axes
        verbose=True,
        dynamic_axes={
            "input_ids": {0: "batch_size"},
            "attn_masks": {0: "batch_size"},
            "token_type_ids": {0: "batch_size"},
            "output": {0: "batch_size"},
        },
        opset_version=10,
        )